In [1]:
include("gates.jl")
using Compat.Test
import QuCircuit: PrimitiveBlock, CompositeBlock, MatrixBlock

In [2]:
GATES = [:X, :Y, :Z]
slower(s::Symbol) = s |> string |> lowercast |> Symbol
for G in GATES
    GATE = Symbol(G, :Gate)
    @eval struct $GATE{MT} <: PrimitiveBlock{1, MT} end
    @eval mat(g::$GATE{MT}) where MT = $(Symbol(:PAULI_,G))
end

In [18]:
############ General Single Qubit Multi-Control Gate ###########
struct ControlBlock{GT<:MatrixBlock, N, MT} <: CompositeBlock{N, MT}
    target::GT
    cbits::Vector{Int}
    ibit::Int
end

struct SingleControlBlock{GT<:MatrixBlock, N, MT} <: CompositeBlock{N, MT}
    target::GT
    cbit::Int
    ibit::Int
end

blocks(cb::SingleControlBlock) = [rb.target]

blocks (generic function with 2 methods)

In [24]:
function mat(cb::ControlBlock{GT, N, MT}) where {GT, N, MT}
   controlled_U1(N, mat(cb.target), cb.cbits, cb.ibit)
end

mat (generic function with 11 methods)

In [25]:
for (G, MATFUNC) in zip(GATES, [:cxgate, :cygate, :czgate])
    GATE = Symbol(G, :Gate)
    @eval function mat(cb::SingleControlBlock{$GATE, N, MT}) where {N, MT}
        $MATFUNC(MT, N, cb.cbit, cb.ibit)
    end
end

In [26]:
#function apply!(r::Register, c::RepeatedBlock{N, MT, XGate{MT}}) where {N, MT}
#end
xg = XGate{ComplexF64}()
mat(xg)

2×2 PermuteMultiply{Complex{Int64},Int64}:
  0     1+0im
 1+0im   0   

In [27]:
cb = SingleControlBlock{XGate, 2, ComplexF64}(xg, 2,1)

SingleControlBlock{XGate,2,Complex{Float64}}(XGate{Complex{Float64}}(), 2, 1)

In [28]:
@test mat(cb) == CNOT_MAT

Test Passed


In [29]:
mutable struct RepeatedBlock{GT<:MatrixBlock, N, MT} <: CompositeBlock{N, MT}
    unit::GT
    bits::Vector{Int}
end
for (G, MATFUNC) in zip(GATES, [:xgate, :ygate, :zgate])
    GGate = Symbol(G, :Gate)
    @eval function mat(cb::RepeatedBlock{$GGate, N, MT}) where {N, MT}
        $MATFUNC(MT, N, cb.bits)
    end
end
blocks(rb::RepeatedBlock) = [rb.unit]

blocks (generic function with 2 methods)

In [12]:
rb = RepeatedBlock{XGate, 2, Complex128}(xg, [1,2])

RepeatedBlock{XGate,2,Complex{Float64}}(XGate{Complex{Float64}}(), [1, 2])

In [13]:
@test mat(rb) == kron(PAULI_X, PAULI_X)

Test Passed


In [23]:
mcb = ControlBlock{XGate, 2, Complex128}(xg, [2], 1)
mat(mcb.target)

2×2 PermuteMultiply{Complex{Int64},Int64}:
  0     1+0im
 1+0im   0   

In [30]:
mat(mcb) == CNOT_MAT

true